<a href="https://colab.research.google.com/github/fbeilstein/machine_learning/blob/master/workbook_06_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exploring simple gradient descent

Explore gradient descent with the following code.
You should understand how it works and its weaknesses.
If you run out of ideas, use the following parameters

function_name | x_ini | y_ini | max_iter | theta | comment
---|---|---|---|---|---
```x**2 + y**2``` | -5.0 | -4.0 | 20 | 0.1 | good convex function
```x**2 + 9*y**2``` | -5.0 | -4.0 | 20 | 0.1 | convex function, see oscillations
```x**2 - y**2``` | 1.5 | 0.0 | 50 | 0.25 | stucks in inflection point
```(x/2)**4+(y/2)**4``` | -1.0 | -8.0 | 30 | 0.1 | divergence
```(x/2)**4+(y/2)**4``` | -1.0 | -1.0 | 50 | 0.1 | very slow convergence
```sin(x) + sin(y)``` | 1.0 | 1.0 | 10 | 0.01 | many global minima
```(1-x**2)+100*(y-x**2)**2``` | -1.0 | -1.0 | 10 | 0.01 | famous Rosenbrock function

In [38]:
#@title #Exploring simple gradient descent


from IPython.display import display
import ipywidgets as widgets


class Memory:
  def __init__(self):
    self.x_ini = 0.0
    self.y_ini = 0.0
    self.iters = 2
    self.theta = 0.5

current_memory = Memory()

button = widgets.Button(description="Recalculate")
iters = widgets.IntSlider(min=2, max=50, 
                          value=current_memory.iters,
                          description='Max Iterations:')

fnc = widgets.Text(value='(x/2)**4+(y/2)**4',
                   #placeholder='function',
                   description='Function:',
                   disabled=False)
theta_widget = widgets.FloatSlider(min=0.0, max=1.0,
                                   value=current_memory.theta,
                                   description='Theta')
def iter_assign(val): 
  current_memory.iters = val.new
def theta_assign(val): 
  current_memory.theta = val.new
iters.observe(iter_assign, names='value')
theta_widget.observe(theta_assign, names='value')

display(widgets.HBox([button, fnc, iters, theta_widget]))

def on_button_clicked(b):
  function = fnc.value
  from IPython.display import clear_output
  clear_output()
  display(widgets.HBox([button, fnc, iters, theta_widget]))
  print('Doing Science...')

  x_ini = current_memory.x_ini
  y_ini = current_memory.y_ini
  max_iter = current_memory.iters


  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  import numpy as np
  from scipy.optimize import minimize

  from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
  transformations = (standard_transformations + (implicit_multiplication_application,))
  f = parse_expr(function, transformations=transformations)

  from sympy import diff
  #g_x = lambda x,y: diff(f, 'x').evalf(subs={'x':x, 'y':y})
  #g_y = lambda x,y: diff(f, 'y').evalf(subs={'x':x, 'y':y})
  #jacobian = lambda x: np.array([g_x(x[0], x[1]), g_y(x[0], x[1])], dtype=float)
  g_x = lambdify(['x', 'y'], diff(f, 'x'), "numpy")
  g_y = lambdify(['x', 'y'], diff(f, 'y'), "numpy")
  jacobian = lambda x,y: np.array([g_x(x, y), g_y(x, y)], dtype=float)


  #func_to_minimize = lambda x: float(f.evalf(subs={'x':x[0], 'y':x[1]}))
  func_to_minimize = lambdify(['x', 'y'], f, "numpy")
  history = [np.array([x_ini, y_ini])]
  crashed = False

  for i in range(max_iter):
    try:
      h = jacobian(history[-1][0], history[-1][1])
    except:
      crashed = True
      break
    x_new = history[-1][0] - current_memory.theta * h[0]
    y_new = history[-1][1] - current_memory.theta * h[1]
    history.append(np.array([x_new, y_new], dtype='float'))


  _h = np.array(history)
  _x = _h.T[0]
  _y = _h.T[1]
  _z = np.array([func_to_minimize(x[0], x[1]) for x in history], dtype=float)

  x_min = min(-10.0, np.min(_x))
  x_max = max(10.0, np.max(_x))
  y_min = min(-10.0, np.min(_y))
  y_max = max(10.0, np.max(_y))

  x_ = np.linspace(x_min, x_max, num=50)
  y_ = np.linspace(y_min, y_max, num=50)
  z_ = np.array([[func_to_minimize(x,y) for x in x_] for y in y_], dtype=float)


  import matplotlib.pyplot as plt
  import base64
  import io
  fig = plt.figure(figsize=(20,20))
  ax = fig.gca()
  ax.axis('off')
  if np.max(z_) - np.min(z_) > 1000:
    ax.contourf(x_, y_, np.sign(z_)*np.log(np.abs(z_) + 1.0), alpha=0.3)  
  else:
    ax.contourf(x_, y_, z_, alpha=0.3)
  plt.close(fig)
  buf = io.BytesIO()
  fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0);
  image_base64 = u'data:image/  png;base64,' + base64.b64encode(buf.getvalue()).decode('utf-8').replace('\n', '')
  buf.close()


  fig = dict(
      layout = dict(
          width=1200, height=600, autosize=False,
          showlegend = False,
          scene = {'domain': { 'x': [0.0, 0.44], 'y': [0, 1] } },
          xaxis1 = {'domain': [0.55, 1], 'range': [x_min, x_max], 'fixedrange': True},
          yaxis1 = {'domain': [0.0, 1.0], 'range': [y_min, y_max], 'fixedrange': True},
          title  = 'Minimization',
          margin = {'t': 50, 'b': 50, 'l': 50, 'r': 50},
          updatemenus = [{'buttons': [{'args': [[k for k in range(len(_h)-1)],
                                                {'frame': {'duration': 500.0, 'redraw': True},
                                                'fromcurrent': False, 'transition': {'duration': 0, 'easing': 'linear'}}],
                                      'label': 'Play',
                                      'method': 'animate'},
                                      {'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                                        'mode': 'immediate',
                                                        'transition': {'duration': 0}}
                                                ],
                                      'label': 'Pause',
                                      'method': 'animate'
                                      }
                                      ],
                          'direction': 'left',
                          'pad': {'r': 10, 't': 85},
                          'showactive': True,
                          'type': 'buttons', 'x': 0.1, 'y': 0, 'xanchor': 'right', 'yanchor': 'top'}],
          sliders = [{'yanchor': 'top',
                      'xanchor': 'left',
                      'currentvalue': {'font': {'size': 16}, 'prefix': 'Step: ', 'visible': True, 'xanchor': 'right'},
                      'transition': {'duration': 0.0},
                      'pad': {'b': 10, 't': 50},
                      'len': 0.9,
                      'x': 0.1,
                      'y': 0,
                      'steps': [{'args': [[k], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': True},
                                                'transition': {'duration': 0, 'easing': 'linear'}}
                                          ],
                                'label': k,
                                'method': 'animate'} for k in range(len(_h)-1)
                      ]}],
          images = [{'source' : image_base64,
                    'xref': 'x', 'yref': 'y',
                    'sizing': 'stretch',
                    'sizex': x_max - x_min, 'sizey': y_max - y_min,
                    'layer': 'below', 'opacity':1.0,
                    'x': x_min, 'y': y_max}]
      ),
      data = [
          {'type': 'scatter3d', 'name': 's3', 'x': _x, 'y': _y, 'z': _z, 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
          {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
          #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
          {'type': 'scatter', 'name': 's2', 'x': _x, 'y': _y, 
          'line': {'color': 'red', 'width': 2}
          },
      ],
      frames=[
          {'name': str(k),
          'data': [
            {'type': 'scatter3d', 'name': 's3', 'x': _x[:k], 'y': _y[:k], 'z': _z[:k], 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
            {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
            #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
            {'type': 'scatter', 'name': 's2', 'x': _x[:k], 'y': _y[:k], 
            'line': {'color': 'red', 'width': 2}
            },
          ]} for k in range(len(_h)-1) ]
  )
  #plot(fig, auto_open=False)
  clear_output()
  if (crashed):
      print("Cannot find inverse matrix, Method compromised")
      print("Current gradient value: ")
      print(h)
      print("Current point: ", history[-1])

  display(widgets.HBox([button, fnc, iters, theta_widget]))
  f = go.Figure(fig)
  f.show()

  def save_pos(pos):
    global current_memory
    current_memory.x_ini = (x_max - x_min) * pos[0] + x_min
    current_memory.y_ini = (y_max - y_min) * (1.0 - pos[1]) + y_min

  main_str = '''
  <canvas id="paint_here"
          onmousedown="mdown_handle(event)"
          onmousemove="mmove_handle(event)"
          onmouseup="mup_handle(event)"></canvas>
  <script>

  var el = document.getElementsByClassName("layer-subplot")[0];
  var rect = el.getBoundingClientRect();

  var canvas = document.getElementById("paint_here");
  canvas.style.cssText = "position:absolute; top:" + rect.top
                      + "px; left: " + rect.left
                      + "px; width:" + rect.width
                      + "px; height:" + rect.height
                      + "px; z-index:1000;";
  canvas.width = rect.width;
  canvas.height = rect.height;
  var ctx = canvas.getContext('2d');
  ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
  //ctx.fillStyle="#00FF00";
  //ctx.fillRect(0, 0, canvas.width, canvas.height); // field
  ''' + 'var x_ini = ' + str((current_memory.x_ini - x_min)/(x_max - x_min)) + ';' + 'var y_ini = ' + str(1.0 - (current_memory.y_ini - y_min)/(y_max - y_min)) + ';' + '''
  var active_pt = [canvas.width * x_ini, canvas.height * y_ini];

  function draw() {
      ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
      //ctx.fillText("drawing", 20, 20);

      ctx.beginPath();
      ctx.arc(active_pt[0], active_pt[1], 10, 0.0, 2.0 * Math.PI, 0);
      ctx.fillStyle = "rgba(210, 0, 0, 0.75)";
      ctx.fill();
  }

  var do_move = false;

  function is_close(pt1, pt2) {
    return   (pt1[0] - pt2[0])*(pt1[0] - pt2[0])
          +  (pt1[1] - pt2[1])*(pt1[1] - pt2[1])
          <= 10*10;
  }

  function mdown_handle(evt) {
    x = evt.offsetX;
    y = evt.offsetY;
    do_move = is_close(active_pt, [x, y]);
  }
      
  function mmove_handle(evt) {
    if (!do_move)
        return;
    active_pt[0] = evt.offsetX;
    active_pt[1] = evt.offsetY;
  }
      
  function mup_handle(evt) {
    do_move = false;
    remember();
  }

  var w = canvas.width;
  var h = canvas.height;

  async function remember() {
    var x = active_pt[0] / w;
    var y = active_pt[1] / h;
    const result = await google.colab.kernel.invokeFunction('notebook.rememberPos', [[x, y]], {});
  }

  var timer = setInterval(draw, 10);

  </script>
  '''

  import IPython
  from google.colab import output
  display(IPython.display.HTML(main_str))
  output.register_callback('notebook.rememberPos', save_pos)


button.on_click(on_button_clicked)

#Exploring sliced function approximation

Following code shows function sliced by a vertical plane.
You can vary point, through which the plane passes and angle of the plane (see x_ini, y_ini and angle slider).
Plot on the right shows slice of the function in red.
Blue curve is the second-order approximation of the slice.
$$
\bar{f}(\alpha)=f(\boldsymbol {x})+\alpha \nabla f(\boldsymbol {x})\boldsymbol {h} +\frac{\alpha^2}{2!}\boldsymbol {h}^{\top} \boldsymbol {H} \boldsymbol {h}.
$$

We can choose gradient descent step based on the minimum of the approximation of the slice (remember, you cannot calculate the whole function for many-dimensional case).
Direction of the slice should be determined by the gradient, but current demo allows you to explore them all.


* Try different function and gain intuition on how good they can be approximated.
* Consider the functions you explored with simple gradient descent.
Understand which problems of the gradient descent can be solved this way.


In [42]:
#@title #Geometric interpretation
function = 'sin(x/3)*sin(y/4)' #@param {type:"string"}

import numpy as np

from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
transformations = (standard_transformations + (implicit_multiplication_application,))
f = parse_expr(function, transformations=transformations)

from sympy import diff
from sympy import lambdify
#g_x = lambda x,y: diff(f, 'x').evalf(subs={'x':x, 'y':y})
#g_y = lambda x,y: diff(f, 'y').evalf(subs={'x':x, 'y':y})
g_x = lambdify(['x', 'y'], diff(f, 'x'), "numpy")
g_y = lambdify(['x', 'y'], diff(f, 'y'), "numpy")
jacobian = lambda x,y: np.array([g_x(x, y), g_y(x, y)], dtype=float)

#g_xx = lambda x,y: diff(diff(f, 'x'), 'x').evalf(subs={'x':x, 'y':y})
#g_xy = lambda x,y: diff(diff(f, 'x'), 'y').evalf(subs={'x':x, 'y':y})
#g_yy = lambda x,y: diff(diff(f, 'y'), 'y').evalf(subs={'x':x, 'y':y})
g_xx = lambdify(['x', 'y'], diff(diff(f, 'x'), 'x'), "numpy")
g_xy = lambdify(['x', 'y'], diff(diff(f, 'x'), 'y'), "numpy")
g_yy = lambdify(['x', 'y'], diff(diff(f, 'y'), 'y'), "numpy")
hessian = lambda x,y: [[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]]


#func_to_minimize = lambda x: f.evalf(subs={'x':x[0], 'y':x[1]})
func_to_minimize = lambdify(['x', 'y'], f, "numpy")

x_min = -10.0
x_max =  10.0
y_min = -10.0
y_max =  10.0

x_ = np.linspace(x_min, x_max, num=50)
y_ = np.linspace(y_min, y_max, num=50)

G = np.array([[jacobian(x,y) for x in x_] for y in y_])
G = np.transpose(G, (2,0,1)).tolist()

H = np.array([[hessian(x,y) for x in x_] for y in y_])
H = np.transpose(H, (2,3,0,1)).tolist()

F = [[func_to_minimize(x,y) for x in x_] for y in y_]

z_max = np.max(F)
z_min = np.min(F)

if z_max - z_min > 1000:
  F2D = [[np.log(np.abs(e)+1.0)*np.sign(e) for e in row] for row in F]
else:
  F2D = F


main_str = '''
<html>

<head>
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script> 

<script type="application/javascript">

var xs = ''' + str(x_.tolist()) + ''';
var ys = ''' + str(y_.tolist()) + ''';
var x_min = ''' + str(x_min) + ''';
var y_min = ''' + str(y_min) + ''';
var z_min = ''' + str(z_min) + ''';
var x_max = ''' + str(x_max) + ''';
var y_max = ''' + str(y_max) + ''';
var z_max = ''' + str(z_max) + ''';
var F2D = ''' + str(F2D) + ''';
var F = ''' + str(F) + ''';
var G = ''' + str(G) + ''';
var H = ''' + str(H) + ''';




function plot_left_func() {
  var cell_for_plot = document.getElementById("for_left_plot");

  var data = [{
    x: xs,
    y: ys,
    z: F2D, 
    showscale: false, 
    opacity: 0.9, 
    type: 'contour'
  }];

  var layout = {
      title: 'Function',
      xaxis: {title: 'x'},
      yaxis: {title: 'y'},
      autosize: false,
      xaxis: {'domain': [0.0, 1.0], 'range': [x_min, x_max], 'fixedrange': true},
      yaxis: {'domain': [0.0, 1.0], 'range': [y_min, y_max], 'fixedrange': true},
      width: 600,
      height: 600,
      showlegend: false,
      margin: {l: 50, r: 50, b: 50, t: 50}
    };
  Plotly.newPlot(cell_for_plot, data, layout);
}



function plot_right_func() {
  var canvas = document.getElementById("paint_left");
  var w = canvas.width;
  var h = canvas.height;
  var plot_x1 = x_min + left_point[0] / w * (x_max - x_min);
  var plot_y1 = y_min + (h - left_point[1]) / h * (y_max - y_min);
  var plot_x2 = x_min + right_point[0] / w * (x_max - x_min);
  var plot_y2 = y_min + (h - right_point[1]) / h * (y_max - y_min);

  var data = [{
    type: 'surface', 
    x: xs,
    y: ys, 
    z: F, 
    opacity: 0.8, 
    showscale: false,
    scene: 'scene',
    colorscale: 'Bluered'
  },{type: 'mesh3d', 
    alphahull: 1.0
  },{
    type: 'scatter3d',
  },{
    type: 'scatter3d',
  },{
    type: 'scatter3d',
  }];
   
  var cell_for_plot = document.getElementById("for_right_plot");
  var layout = {
      //title: 'Function',
      scene: {
        xaxis: {title: 'x'},
        yaxis: {title: 'y'},
        autosize: true,
        width: 600,
        height: 600,
        margin: {l: 0, r: 0, b: 0, t: 0, pad: 0}
      },
      showlegend: false,
    };
Plotly.newPlot(cell_for_plot, data, layout);
}

function approximate(x, y, F) {
    var x_iters = xs.length;
    var y_iters = ys.length;
    var idx_xl = Math.floor((x - x_min)/(x_max - x_min)*x_iters);
    var idx_yl = Math.floor((y - y_min)/(y_max - y_min)*y_iters);
    var idx_xu = idx_xl + 1;
    var idx_yu = idx_yl + 1;
    if (idx_xu >= xs.length)
      idx_xu = idx_xl;
    if (idx_yu >= ys.length)
      idx_yu = idx_yl;


    var F_l = 0.0;
    var F_u = 0.0;
    if (idx_xl == idx_xu) {
      F_l = F[idx_yl][idx_xl];
      F_u = F[idx_yu][idx_xl];
    } else {
      F_l = F[idx_yl][idx_xl] * (xs[idx_xu] - x) / (xs[idx_xu] - xs[idx_xl])
          + F[idx_yl][idx_xu] * (x - xs[idx_xl]) / (xs[idx_xu] - xs[idx_xl]);
      F_u = F[idx_yu][idx_xl] * (xs[idx_xu] - x) / (xs[idx_xu] - xs[idx_xl])
          + F[idx_yu][idx_xu] * (x - xs[idx_xl]) / (xs[idx_xu] - xs[idx_xl]);
    }
    var F_ = 0;
    if (idx_yl == idx_yu)
      F_ = F_l;
    else
      F_ = F_l * (ys[idx_yu] - y) / (ys[idx_yu] - ys[idx_yl])
         + F_u * (y - ys[idx_yl]) / (ys[idx_yu] - ys[idx_yl]);
    return F_;
}

var Gx = 0.0;
var Gy = 0.0;

function update_3d() {
  var cell_for_plot = document.getElementById("for_right_plot");
  Plotly.deleteTraces(cell_for_plot, [1,2,3,4]);
  var canvas = document.getElementById("paint_left");
  var w = canvas.width;
  var h = canvas.height;
  var plot_x1 = x_min + left_point[0] / w * (x_max - x_min);
  var plot_y1 = y_min + (h - left_point[1]) / h * (y_max - y_min);
  var plot_x2 = x_min + right_point[0] / w * (x_max - x_min);
  var plot_y2 = y_min + (h - right_point[1]) / h * (y_max - y_min);
  //console.log(plot_x1, plot_y1, plot_x2, plot_y2);

  var approx_x = x_min + middle_point[0] / w * (x_max - x_min);
  var approx_y = y_min + (h - middle_point[1]) / h * (y_max - y_min);

/*
      var y_ = ys[j] - y;
      var row = [];
      for (var i = 0; i < xs.length; ++i) {
        var x_ = xs[i] - x;
        var approx = f + (g[0]*x_ + g[1]*y_) + ((h[0][0]*x_ + h[0][1]*y_)*x_ + (h[1][0]*x_ + h[1][1]*y_)*y_) / 2.0;
        row.push(approx);

*/

  var median_x = x_min + middle_point[0] / w * (x_max - x_min);
  var median_y = y_min + (h - middle_point[1]) / h * (y_max - y_min);
  medF = approximate(median_x, median_y, F);
  Gx = approximate(median_x, median_y, G[0]);
  Gy = approximate(median_x, median_y, G[1]);
  H00 = approximate(median_x, median_y, H[0][0]);
  H01 = approximate(median_x, median_y, H[0][1]);
  H10 = approximate(median_x, median_y, H[1][0]);
  H11 = approximate(median_x, median_y, H[1][1]);


  var x_iters = xs.length;
  var y_iters = ys.length;
  var m_iters = Math.max(x_iters, y_iters);
  var v = [plot_x2 - plot_x1, plot_y2 - plot_y1];
  var l = Math.sqrt(v[0]*v[0] + v[1]*v[1]);
  v = [v[0]/m_iters, v[1]/m_iters];
  x_ = [];
  y_ = [];
  z_ = [];
  Z_ = [];
  for (var i = 0; i < m_iters; ++i) {
    var x = i*v[0] + plot_x1;
    var y = i*v[1] + plot_y1;
    x_.push(x);
    y_.push(y);
    var F_ = approximate(x, y, F);
    z_.push(F_);
    var dx = x - median_x;
    var dy = y - median_y;
    var approx = medF + (Gx*dx + Gy*dy) + ((H00*dx + H01*dy)*dx + (H10*dx + H11*dy)*dy) / 2.0;
    if (approx > z_min && approx < z_max)
      Z_.push(approx);
    else
      Z_.push(null);
  }


  var data = [{
    type: 'mesh3d', 
    alphahull:1.0,
    //delaunayaxis: [plot_y1-plot_y2, plot_x2-plot_x1, 0.0],
    x: [plot_x1, plot_x1+0.001*(x_max-x_min), plot_x2,  plot_x2, ], 
    y: [plot_y1, plot_y1-0.001*(y_max-y_min), plot_y2,  plot_y2, ], 
    z: [z_min,   z_max,                       z_max*0.99, z_min],
    i: [0, 0, 0, 1],
    j: [1, 2, 3, 2],
    k: [2, 3, 1, 3],
    color:'rgba(0,160,0,1.0)', 
    opacity: 0.1, 
    flatshading: true,
    //showscale: false
  },{
    type: 'scatter3d',
    mode: 'lines',
    x: x_,
    y: y_,
    z: z_,
    color: 'rgba(0,160,0,0.8)',
    line: {color: 'rgba(0,160,0,0.8)', width: 4}
  },{
    type: 'scatter3d',
    mode: 'lines',
    x: [median_x, median_x],
    y: [median_y, median_y],
    z: [z_min, z_max],
    color: 'rgba(0,0,160,0.8)',
    line: {color: 'rgba(0,0,160,0.8)', width: 4}
  },{
    type: 'scatter3d',
    mode: 'lines',
    x: x_,
    y: y_,
    z: Z_,
    color: 'rgba(160,0,0,0.8)',
    line: {color: 'rgba(160,0,0,0.8)', width: 4}
  }];
  Plotly.addTraces(cell_for_plot, data);
}



    var active_state = 0; // 0 -none, 1 - left, 2 - right, 3 - center
    var left_point = [0.0, 0.0];
    var right_point = [0.0, 0.0];
    var middle_point = [0.0, 0.0];

    function draw() {
        var plt_div = document.getElementById('for_left_plot');
  var el = plt_div.getElementsByClassName('plot')[0];
  //el = el.getChildren()[0];
  var rect = el.getBoundingClientRect();
  var canvas = document.getElementById("paint_left");
  canvas.style.cssText = "position:absolute; top:" + rect.top
                      + "px; left: " + rect.left
                      + "px; width:" + rect.width
                      + "px; height:" + rect.height
                      + "px; z-index:1000;";
  canvas.width = rect.width;
  canvas.height = rect.height;
  var w = canvas.width;
  var h = canvas.height;


      var canvas = document.getElementById("paint_left");
      var ctx = canvas.getContext('2d');
      ctx.clearRect(0, 0, w, h);
      ctx.strokeStyle = "rgb(0,120,0)";
      ctx.lineWidth = 3;
      ctx.globalAlpha = 1.0;
      //ctx.setLineDash([5, 15]);
      ctx.beginPath();
      ctx.moveTo(left_point[0], left_point[1]);
      ctx.lineTo(right_point[0], right_point[1]);
      ctx.stroke();
      ctx.globalAlpha = 1.0;

      ctx.beginPath();
      ctx.arc(middle_point[0], middle_point[1], 10, 0.0, 2.0 * Math.PI, 0);
      ctx.fillStyle = "rgba(210, 0, 0, 0.75)";
      ctx.fill();


      ctx.strokeStyle = "rgb(0,0,0)";
      ctx.lineWidth = 2;
      for (var i = 0; i < xs.length; i += 5) {
        for (var j = 0; j < ys.length; j += 5) {
          canvas_arrow(ctx, (xs[i] - x_min)/(x_max-x_min)*w, 
                            (1.0 - (ys[j] - y_min)/(y_max-y_min))*h, 
                            G[0][j][i]/(x_max-x_min)*w*5.0, 
                           -G[1][j][i]/(y_max-y_min)*h*5.0);
        }
      }

      ctx.strokeStyle = "rgb(255,0,0)";
      ctx.lineWidth = 2;
      canvas_arrow(ctx, middle_point[0], 
                        middle_point[1], 
                        Gx/(x_max-x_min)*w*5.0, 
                       -Gy/(y_max-y_min)*h*5.0);


      //ctx.fillStyle="#00FF00";
      //ctx.fillRect(0, 0, canvas.width, canvas.height); // field
    }


function canvas_arrow(context, fromx, fromy, dx, dy) {
    context.beginPath();
    var headlen = 10; // length of head in pixels
    var tox = fromx + dx;
    var toy = fromy + dy;
    var angle = Math.atan2(dy, dx);
    context.moveTo(fromx, fromy);
    context.lineTo(tox, toy);
    context.lineTo(tox - headlen * Math.cos(angle - Math.PI / 6), toy - headlen * Math.sin(angle - Math.PI / 6));
    context.moveTo(tox, toy);
    context.lineTo(tox - headlen * Math.cos(angle + Math.PI / 6), toy - headlen * Math.sin(angle + Math.PI / 6));
    context.stroke();
  }



  function mdown_handle(evt) {
    var x = evt.offsetX;
    var y = evt.offsetY;
    var vec = [left_point[0] - right_point[0], left_point[1] - right_point[1]];
    var lng = Math.sqrt(vec[0]*vec[0] + vec[1]*vec[1]);
    vec = [vec[0] / lng, vec[1] / lng];
    var u = [x - right_point[0], y - right_point[1]];
    var proj_paral = u[0] * vec[0] + u[1] * vec[1];
    var proj_perp = Math.sqrt(u[0]*u[0] + u[1]*u[1] - proj_paral*proj_paral);
    if (proj_perp > 10.0)
      return;
    var dm = [x - middle_point[0], y - middle_point[1]];
    if (dm[0]*dm[0] + dm[1]*dm[1] < 100) {
      active_state = 3;
      draw();
      return;
    }
    var ml = [left_point[0] - middle_point[0], left_point[1] - middle_point[1]];
    //var dl = [left_point[0] - x, left_point[1] - y];
    //if (dl[0]*dl[0] + dl[1]*dl[1] < ml[0]*ml[0] + ml[1]*ml[1])
    //  active_state = 2;
    //else
    //  active_state = 1;
    var lng = Math.sqrt(ml[0]*ml[0] + ml[1]*ml[1]);

    proj_paral = Math.abs(proj_paral);
    if (proj_paral < lng)
      active_state = 2;
    if (proj_paral > lng)
      active_state = 1;
    //if (proj_paral > lng / 3.0 && proj_paral < 2.0 * lng / 3.0)
    //  active_state = 3;
    draw();
  }
      
  function cross_point(A, B, w, h) {
    var u = [A[0] / w, A[1] / h];
    var v = [(B[0] - A[0]) / w, (B[1] - A[1]) / h];
    var lambda_1 = 1.0;
    if (v[0]*v[0] > 0.00001) {
      lambda_1 = Math.max(-u[0] / v[0], (1-u[0]) / v[0]);
    } else {
      lambda_1 = Infinity;
    }
    var lambda_2 = 1.0;
    if (v[1]*v[1] > 0.00001) {
      lambda_2 = Math.max(-u[1] / v[1], (1-u[1]) / v[1]);
    } else {
      lambda_2 = Infinity;
    }
    var lambda = Math.min(lambda_1, lambda_2);
    var p = [u[0] + lambda * v[0], u[1] + lambda * v[1]];
    return [p[0] * w, p[1] * h];
  }

  function restrict_point(pt, w, h) {
    pt = [Math.floor(pt[0]), Math.floor(pt[1])];
    if (pt[0] <= 0)
      pt[0] = 1;
    if (pt[0] >= w)
      pt[0] = w - 1;
    if (pt[1] <= 0)
      pt[1] = 1;
    if (pt[1] >= h)
      pt[1] = h - 1;
    return pt;
  }

  function mmove_handle(evt) {
    if (active_state == 0)
        return;
    var x = evt.offsetX;
    var y = evt.offsetY;
    var canvas = document.getElementById("paint_left");
    var w = canvas.width;
    var h = canvas.height;
    
    var dm = [x - middle_point[0], y - middle_point[1]];
    if (dm[0]*dm[0] + dm[1]*dm[1] < 100) {
      active_state = 3;
    }

    if (active_state == 1) {
      left_point = cross_point(middle_point, [x, y], w, h);
      right_point = cross_point(left_point, middle_point, w, h);
    }
    if (active_state == 2) {
      right_point = cross_point(middle_point, [x, y], w, h);
      left_point = cross_point(right_point, middle_point, w, h);
    }
    if (active_state == 3) {
      //var delta = [x - middle_point[0], y - middle_point[1]];
      middle_point = [x, y];
      var v = [left_point[0] - right_point[0], left_point[1] - right_point[1]];
      left_point = cross_point([x, y], [v[0]+x, v[1]+y], w, h);
      right_point = cross_point([x, y], [-v[0]+x, -v[1]+y], w, h);
//      left_point = cross_point(middle_point, [left_point[0]+delta[0], left_point[1]+delta[1]], w, h);
//      right_point = cross_point(middle_point, [right_point[0]+delta[0], right_point[1]+delta[1]], w, h);
    }

    left_point = restrict_point(left_point, w, h);
    right_point = restrict_point(right_point, w, h);
    middle_point = restrict_point(middle_point, w, h);



    //if (active_state == 1) { // move left point
    //  left_point = cross_point(right_point, [x, y], w, h);
    //}
    //if (active_state == 2) {
    //  right_point = cross_point(left_point, [x, y], w, h);
    //}
    //if (active_state == 3) {
    //  var v = [left_point[0] - right_point[0], left_point[1] - right_point[1]];
    //  left_point = cross_point([x, y], [v[0]+x, v[1]+y], w, h);
    //  right_point = cross_point([x, y], [-v[0]+x, -v[1]+y], w, h);
    //}

    console.log(active_state,left_point, middle_point, right_point);

    update_3d();  
    draw();
    
  }
      
  function mup_handle(evt) {
    active_state = 0;
    draw();
  }

  </script>
</head>

<body>
<canvas id="paint_left"
  onmousedown="mdown_handle(event)"
  onmousemove="mmove_handle(event)"
  onmouseup="mup_handle(event)"></canvas>

<div id="everything" style="height:600px; width:1200px; margin: 0px 0px 0px 0px;">
<div id="for_left_plot" > </div>
<div id="for_right_plot" style="height:600px; position:absolute; top:0px; left:600px; margin: 0px 0px 0px 0px;"> </div>
</div>

</body>

<script>
function init()
{
  plot_left_func();
  plot_right_func();

  var plt_div = document.getElementById('for_left_plot');
  var el = plt_div.getElementsByClassName('plot')[0];
  //el = el.getChildren()[0];
  var rect = el.getBoundingClientRect();
  var canvas = document.getElementById("paint_left");
  canvas.style.cssText = "position:absolute; top:" + rect.top
                      + "px; left: " + rect.left
                      + "px; width:" + rect.width
                      + "px; height:" + rect.height
                      + "px; z-index:1000;";
  canvas.width = rect.width;
  canvas.height = rect.height;
  console.log(rect);
  var ctx = canvas.getContext('2d');
  ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
  left_point = [0.0, canvas.height / 2.0];
  right_point = [canvas.width, canvas.height / 2.0];
  middle_point = [canvas.width / 2.0, canvas.height / 2.0];
  draw();
  update_3d();
  
  //timer = setInterval(draw, 200);
}
init();
</script>

</html>
'''

import IPython
display(IPython.display.HTML(main_str))

#Exploring modified gradient descent

After you gained some intuition with two previous demos, test it on the following.
Try different functions that caused trouble with gradients.
Check, whether choosing a step with a second order approximation fixes them.
Test, whether your intuition is right or wrong telling you if the problem is "fixable".

In [39]:
#@title #Exploring gradient+hessian descent


from IPython.display import display
import ipywidgets as widgets


class Memory:
  def __init__(self):
    self.x_ini = 0.0
    self.y_ini = 0.0
    self.iters = 2

current_memory = Memory()

button = widgets.Button(description="Recalculate")
iters = widgets.IntSlider(min=2, max=50, 
                          value=current_memory.iters,
                          description='Max Iterations:')

fnc = widgets.Text(value='(x/2)**4+(y/2)**4',
                   #placeholder='function',
                   description='Function:',
                   disabled=False)

def set_iter(val):
  current_memory.iters = val.new
iters.observe(set_iter, names='value')

display(widgets.HBox([button, fnc, iters]))

def on_button_clicked(b):
  function = fnc.value
  from IPython.display import clear_output
  clear_output()
  display(widgets.HBox([button, fnc, iters]))
  print('Doing Science...')

  x_ini = current_memory.x_ini
  y_ini = current_memory.y_ini
  max_iter = current_memory.iters


  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  import numpy as np
  from scipy.optimize import minimize

  from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
  transformations = (standard_transformations + (implicit_multiplication_application,))
  f = parse_expr(function, transformations=transformations)

  from sympy import diff
  from sympy import lambdify
  #g_x = lambda x,y: diff(f, 'x').evalf(subs={'x':x, 'y':y})
  #g_y = lambda x,y: diff(f, 'y').evalf(subs={'x':x, 'y':y})
  g_x = lambdify(['x', 'y'], diff(f, 'x'), "numpy")
  g_y = lambdify(['x', 'y'], diff(f, 'y'), "numpy")
  jacobian = lambda x: np.array([g_x(x[0], x[1]), g_y(x[0], x[1])], dtype=float)


  #g_xx = lambda x,y: diff(diff(f, 'x'), 'x').evalf(subs={'x':x, 'y':y})
  #g_xy = lambda x,y: diff(diff(f, 'x'), 'y').evalf(subs={'x':x, 'y':y})
  #g_yy = lambda x,y: diff(diff(f, 'y'), 'y').evalf(subs={'x':x, 'y':y})
  g_xx = lambdify(['x', 'y'], diff(diff(f, 'x'), 'x'), "numpy")
  g_xy = lambdify(['x', 'y'], diff(diff(f, 'x'), 'y'), "numpy")
  g_yy = lambdify(['x', 'y'], diff(diff(f, 'y'), 'y'), "numpy")  
  H = lambda x,y: [[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]]

  hessian = lambda x: np.array(H(x[0], x[1]), dtype=float)

  #func_to_minimize = lambda x: float(f.evalf(subs={'x':x[0], 'y':x[1]}))
  func_to_minimize = lambdify(['x', 'y'], f, "numpy")  

  history = [np.array([x_ini, y_ini])]
  crashed = False

  for i in range(max_iter):
    h = jacobian(history[-1])
    H_ = hessian(history[-1])
    if -1E-20 < np.dot(h, H_.dot(h)) < 1E-20:
      crashed = True
      break
    else:
      theta = np.dot(h, h) / np.dot(h, H_.dot(h))
    x_new = history[-1][0] - theta * h[0]
    y_new = history[-1][1] - theta * h[1]
    history.append(np.array([x_new, y_new], dtype='float'))


  _h = np.array(history)
  _x = _h.T[0]
  _y = _h.T[1]
  _z = np.array([func_to_minimize(x[0], x[1]) for x in history], dtype=float)

  x_min = min(-10.0, np.min(_x))
  x_max = max(10.0, np.max(_x))
  y_min = min(-10.0, np.min(_y))
  y_max = max(10.0, np.max(_y))

  x_ = np.linspace(x_min, x_max, num=50)
  y_ = np.linspace(y_min, y_max, num=50)
  z_ = np.array([[func_to_minimize(x,y) for x in x_] for y in y_], dtype=float)


  import matplotlib.pyplot as plt
  import base64
  import io
  fig = plt.figure(figsize=(20,20))
  ax = fig.gca()
  ax.axis('off')
  if np.max(z_) - np.min(z_) > 1000:
    ax.contourf(x_, y_, np.sign(z_)*np.log(np.abs(z_) + 1.0), alpha=0.3)  
  else:
    ax.contourf(x_, y_, z_, alpha=0.3)
  plt.close(fig)
  buf = io.BytesIO()
  fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0);
  image_base64 = u'data:image/  png;base64,' + base64.b64encode(buf.getvalue()).decode('utf-8').replace('\n', '')
  buf.close()


  fig = dict(
      layout = dict(
          width=1200, height=600, autosize=False,
          showlegend = False,
          scene = {'domain': { 'x': [0.0, 0.44], 'y': [0, 1] } },
          xaxis1 = {'domain': [0.55, 1], 'range': [x_min, x_max], 'fixedrange': True},
          yaxis1 = {'domain': [0.0, 1.0], 'range': [y_min, y_max], 'fixedrange': True},
          title  = 'Minimization',
          margin = {'t': 50, 'b': 50, 'l': 50, 'r': 50},
          updatemenus = [{'buttons': [{'args': [[k for k in range(len(_h)-1)],
                                                {'frame': {'duration': 500.0, 'redraw': True},
                                                'fromcurrent': False, 'transition': {'duration': 0, 'easing': 'linear'}}],
                                      'label': 'Play',
                                      'method': 'animate'},
                                      {'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                                        'mode': 'immediate',
                                                        'transition': {'duration': 0}}
                                                ],
                                      'label': 'Pause',
                                      'method': 'animate'
                                      }
                                      ],
                          'direction': 'left',
                          'pad': {'r': 10, 't': 85},
                          'showactive': True,
                          'type': 'buttons', 'x': 0.1, 'y': 0, 'xanchor': 'right', 'yanchor': 'top'}],
          sliders = [{'yanchor': 'top',
                      'xanchor': 'left',
                      'currentvalue': {'font': {'size': 16}, 'prefix': 'Step: ', 'visible': True, 'xanchor': 'right'},
                      'transition': {'duration': 0.0},
                      'pad': {'b': 10, 't': 50},
                      'len': 0.9,
                      'x': 0.1,
                      'y': 0,
                      'steps': [{'args': [[k], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': True},
                                                'transition': {'duration': 0, 'easing': 'linear'}}
                                          ],
                                'label': k,
                                'method': 'animate'} for k in range(len(_h)-1)
                      ]}],
          images = [{'source' : image_base64,
                    'xref': 'x', 'yref': 'y',
                    'sizing': 'stretch',
                    'sizex': x_max - x_min, 'sizey': y_max - y_min,
                    'layer': 'below', 'opacity':1.0,
                    'x': x_min, 'y': y_max}]
      ),
      data = [
          {'type': 'scatter3d', 'name': 's3', 'x': _x, 'y': _y, 'z': _z, 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
          {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
          #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
          {'type': 'scatter', 'name': 's2', 'x': _x, 'y': _y, 
          'line': {'color': 'red', 'width': 2}
          },
      ],
      frames=[
          {'name': str(k),
          'data': [
            {'type': 'scatter3d', 'name': 's3', 'x': _x[:k], 'y': _y[:k], 'z': _z[:k], 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
            {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
            #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
            {'type': 'scatter', 'name': 's2', 'x': _x[:k], 'y': _y[:k], 
            'line': {'color': 'red', 'width': 2}
            },
          ]} for k in range(len(_h)-1) ]
  )
  #plot(fig, auto_open=False)
  clear_output()
  if (crashed):
      print("Cannot find inverse matrix, Method compromised")
      print("Current Hessian value: ")
      print(H_)
      print("Current point: ", history[-1])
      print("np.dot(h, H_.dot(h)): ", np.dot(h, H_.dot(h)))

  display(widgets.HBox([button, fnc, iters]))
  f = go.Figure(fig)
  f.show()

  def save_pos(pos):
    global current_memory
    current_memory.x_ini = (x_max - x_min) * pos[0] + x_min
    current_memory.y_ini = (y_max - y_min) * (1.0 - pos[1]) + y_min

  main_str = '''
  <canvas id="paint_here"
          onmousedown="mdown_handle(event)"
          onmousemove="mmove_handle(event)"
          onmouseup="mup_handle(event)"></canvas>
  <script>

  var el = document.getElementsByClassName("layer-subplot")[0];
  var rect = el.getBoundingClientRect();

  var canvas = document.getElementById("paint_here");
  canvas.style.cssText = "position:absolute; top:" + rect.top
                      + "px; left: " + rect.left
                      + "px; width:" + rect.width
                      + "px; height:" + rect.height
                      + "px; z-index:1000;";
  canvas.width = rect.width;
  canvas.height = rect.height;
  var ctx = canvas.getContext('2d');
  ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
  //ctx.fillStyle="#00FF00";
  //ctx.fillRect(0, 0, canvas.width, canvas.height); // field
  ''' + 'var x_ini = ' + str((current_memory.x_ini - x_min)/(x_max - x_min)) + ';' + 'var y_ini = ' + str(1.0 - (current_memory.y_ini - y_min)/(y_max - y_min)) + ';' + '''
  var active_pt = [canvas.width * x_ini, canvas.height * y_ini];

  function draw() {
      ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
      //ctx.fillText("drawing", 20, 20);

      ctx.beginPath();
      ctx.arc(active_pt[0], active_pt[1], 10, 0.0, 2.0 * Math.PI, 0);
      ctx.fillStyle = "rgba(210, 0, 0, 0.75)";
      ctx.fill();
  }

  var do_move = false;

  function is_close(pt1, pt2) {
    return   (pt1[0] - pt2[0])*(pt1[0] - pt2[0])
          +  (pt1[1] - pt2[1])*(pt1[1] - pt2[1])
          <= 10*10;
  }

  function mdown_handle(evt) {
    x = evt.offsetX;
    y = evt.offsetY;
    do_move = is_close(active_pt, [x, y]);
  }
      
  function mmove_handle(evt) {
    if (!do_move)
        return;
    active_pt[0] = evt.offsetX;
    active_pt[1] = evt.offsetY;
  }
      
  function mup_handle(evt) {
    do_move = false;
    remember();
  }

  var w = canvas.width;
  var h = canvas.height;

  async function remember() {
    var x = active_pt[0] / w;
    var y = active_pt[1] / h;
    const result = await google.colab.kernel.invokeFunction('notebook.rememberPos', [[x, y]], {});
  }

  var timer = setInterval(draw, 10);

  </script>
  '''

  import IPython
  from google.colab import output
  display(IPython.display.HTML(main_str))
  output.register_callback('notebook.rememberPos', save_pos)


button.on_click(on_button_clicked)

#Exploring second-order approximation

Now we are on a way to Newton's method.
Use the following demo to explore a second-order approximation of a function.
Try different functions and points of approximation until you get intuitive feeling how the second order approximation looks like.
You can find some thought-provoking examples in the following table but feel free to try anything that comes to your mind.

function | x_ini | y_ini | Comment
---|---|---|---
sin(x)+4*sin(y) | 2.9 | 4.9 | non-positive definite Hessian
x**2 | 0.0 | 0.0 | long valley, no global minimum
x** 2 - y** 2 | 0.0 | 0.0 | saddle point
x** 4 + y** 4 | 8.0 | 8.0 | convex, non-second-order
x** 4 + y** 4 | 1.0 | 1.0 | convex, non-second-order

On the left you will see eigenvalues of the Hessian matrix.
Gain some intuition how different signs of these eigenvalues depend on function you approximate and how the approximation looks like.

In [47]:
#@title #Geometric interpretation
function = 'x**2+y**2' #@param {type:"string"}

import numpy as np

from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
transformations = (standard_transformations + (implicit_multiplication_application,))
f = parse_expr(function, transformations=transformations)

from sympy import diff
from sympy import lambdify
#g_x = lambda x,y: diff(f, 'x').evalf(subs={'x':x, 'y':y})
#g_y = lambda x,y: diff(f, 'y').evalf(subs={'x':x, 'y':y})
#jacobian = lambda x: np.array([g_x(x[0], x[1]), g_y(x[0], x[1])], dtype=float)
g_x = lambdify(['x', 'y'], diff(f, 'x'), "numpy")
g_y = lambdify(['x', 'y'], diff(f, 'y'), "numpy")
jacobian = lambda x,y: [g_x(x, y), g_y(x, y)]

#g_xx = lambda x,y: diff(diff(f, 'x'), 'x').evalf(subs={'x':x, 'y':y})
#g_xy = lambda x,y: diff(diff(f, 'x'), 'y').evalf(subs={'x':x, 'y':y})
#g_yy = lambda x,y: diff(diff(f, 'y'), 'y').evalf(subs={'x':x, 'y':y})
#H = lambda x,y: [[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]]
#hessian = lambda x: np.array(H(x[0], x[1]), dtype=float)
g_xx = lambdify(['x', 'y'], diff(diff(f, 'x'), 'x'), "numpy")
g_xy = lambdify(['x', 'y'], diff(diff(f, 'x'), 'y'), "numpy")
g_yy = lambdify(['x', 'y'], diff(diff(f, 'y'), 'y'), "numpy")
hessian = lambda x,y: [[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]]

#func_to_minimize = lambda x: f.evalf(subs={'x':x[0], 'y':x[1]})
func_to_minimize = lambdify(['x', 'y'], f, "numpy")


x_min = -10.0
x_max =  10.0
y_min = -10.0
y_max =  10.0

x_ = np.linspace(x_min, x_max, num=30)
y_ = np.linspace(y_min, y_max, num=30)

x_for3d = np.linspace(x_min, x_max, num=15)
y_for3d = np.linspace(y_min, y_max, num=15)


G = [[jacobian(x,y) for x in x_] for y in y_]
H = [[hessian(x,y) for x in x_] for y in y_]
F = [[func_to_minimize(x,y) for x in x_] for y in y_]

F_for3d = [[func_to_minimize(x,y) for x in x_for3d] for y in y_for3d]

z_max = np.max(F)
z_min = np.min(F)

if z_max - z_min > 1000:
  F2D = [[np.log(np.abs(e)+1.0)*np.sign(e) for e in row] for row in F]
else:
  F2D = F

E1 = []
E2 = []
for row in H:
  temp1 = []
  temp2 = []
  for h in row:
    trace = h[0][0] + h[1][1]
    det = np.linalg.det(h)
    delta = trace**2 - 4 * det
    if delta < 0.0:
      delta = 0.0
    temp1.append((trace + np.sqrt(delta)) / 2.0)
    temp2.append((trace - np.sqrt(delta)) / 2.0)
  E1.append(temp1)
  E2.append(temp2)


#G = jacobian([x_ini, y_ini])
#H = hessian([x_ini, y_ini])
#F = func_to_minimize([x_ini, y_ini])
#appr = lambda x,y: F + np.dot(G, np.array([x-x_ini, y - y_ini])) + 0.5 * np.dot(np.array([x-x_ini, y - y_ini]), H.dot(np.array([x-x_ini, y - y_ini])))
#Z_ = np.array([[appr(x, y) for x in x_] for y in y_], dtype=float)


main_str = '''
<html>

<head>
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script> 

<script type="application/javascript">

var xs = ''' + str(x_.tolist()) + ''';
var ys = ''' + str(y_.tolist()) + ''';
var x_min = ''' + str(x_min) + ''';
var y_min = ''' + str(y_min) + ''';
var z_min = ''' + str(z_min) + ''';
var x_max = ''' + str(x_max) + ''';
var y_max = ''' + str(y_max) + ''';
var z_max = ''' + str(z_max) + ''';
var G = ''' + str(G) + ''';
var H = ''' + str(H) + ''';
var F = ''' + str(F) + ''';
var F2D = ''' + str(F2D) + ''';
var E1 = ''' + str(E1) + ''';
var E2 = ''' + str(E2) + ''';
var x_for3d = ''' + str(x_for3d.tolist()) + ''';
var y_for3d = ''' + str(y_for3d.tolist()) + ''';
var F_for3d = ''' + str(F_for3d) + ''';

  function generate_approximation(x_idx, y_idx) {

  }

  function plot_potentials() {
    // get x and y
    var x, y;
    if (active_pt) {
      x = active_pt[0];
      y = active_pt[1];
    } else {
      x = all_pts[0][0];
      y = all_pts[0][1];
    }
    var canvas = document.getElementById("paint");
    var w = canvas.width;
    var h = canvas.height;
    x = (x / w) * (x_max - x_min) + x_min;
    y = (1.0 - y / h) * (y_max - y_min) + y_min;

    // approximate with data available
    var delta = x_max - x_min;
    var x_idx_approx = 0;
    for (var i = 0; i < xs.length; ++i)
      if (Math.abs(xs[i] - x) < delta) {
        delta = Math.abs(xs[i] - x);
        x_idx_approx = i;
      }
    var delta = y_max - y_min;
    var y_idx_approx = 0;
    for (var i = 0; i < ys.length; ++i)
      if (Math.abs(ys[i] - y) < delta) {
        delta = Math.abs(ys[i] - y);
        y_idx_approx = i;
      }

    // get parameters
    g = G[y_idx_approx][x_idx_approx];
    h = H[y_idx_approx][x_idx_approx];
    f = F[y_idx_approx][x_idx_approx];
    e1 = E1[y_idx_approx][x_idx_approx];
    e2 = E2[y_idx_approx][x_idx_approx];

    // calculate approximation
    var F_approx = [];
    for (var j = 0; j < ys.length; ++j) {
      var y_ = ys[j] - y;
      var row = [];
      for (var i = 0; i < xs.length; ++i) {
        var x_ = xs[i] - x;
        var approx = f + (g[0]*x_ + g[1]*y_) + ((h[0][0]*x_ + h[0][1]*y_)*x_ + (h[1][0]*x_ + h[1][1]*y_)*y_) / 2.0;
        if (z_max - z_min > 1000)
          row.push(Math.log(Math.abs(approx)+1.0)*Math.sign(approx));
        else
          row.push(approx);
      }
      F_approx.push(row);
    }

    var F_approx_3d = [];
    for (var j = 0; j < y_for3d.length; ++j) {
      var y_ = y_for3d[j] - y;
      var row = [];
      for (var i = 0; i < x_for3d.length; ++i) {
        var x_ = x_for3d[i] - x;
        var approx = f + (g[0]*x_ + g[1]*y_) + ((h[0][0]*x_ + h[0][1]*y_)*x_ + (h[1][0]*x_ + h[1][1]*y_)*y_) / 2.0;
        row.push(approx);
      }
      F_approx_3d.push(row);
    }


      var data = [{
        'type': 'scatter3d', 
        'mode': 'lines', 
        'name': 's3', 
        'x': [x, x], 
        'y': [y, y], 
        'z': [z_min, z_max], 
        'line': {'color': 'blue', 'width': 3}
      },{
       type: 'surface', 
       x: x_for3d,
       y: y_for3d, 
       z: F_for3d, 
       opacity: 0.8, 
       showscale: false,
       scene: 'scene',
       colorscale: 'Bluered'
      },{
       type: 'surface',
       x: x_for3d, 
       y: y_for3d, 
       z: F_approx_3d,
       opacity: 0.5, 
       showscale: false,
       scene: 'scene',
       colorscale : [[0.0, 'rgb(0, 255, 0)'], [1.0, 'rgb(0, 254, 0)']]
      },{
        x: xs,
        y: ys,
        z: F2D, 
        showscale: false, 
        opacity: 0.9, 
        type: 'contour'
      },{
        x: xs,
        y: ys,
        z: F_approx, 
        showscale: false, 
        opacity: 1.0, 
        type: 'contour',
        line: {
          width: 2,
          color: 'black'
        },
        contours: {
          coloring: 'none',
        }
      }];

      var cell_for_plot = document.getElementById("for_right_plot");

      var layout = {
          title: 'Approximation. Eigenvalues: ' + e1.toFixed(3) + '; ' + e2.toFixed(3),
          scene: {
            domain: { 'x': [0.55, 1.0], 'y': [0.0, 1.0] }, 
            zaxis: {'range': [z_min, z_max]},
            aspectmode: 'cube' 
          },
          xaxis1: {'range': [x_min, x_max], 'domain': [0.0, 0.44], 'fixedrange': true},
          yaxis1: {'range': [y_min, y_max], 'domain': [0.0, 1.0], 'fixedrange': true},
          width: 1200,
          height: 600,
          autosize: false,
          showlegend: false,
          margin: {l: 50, r: 50, b: 50, t: 50}
        };
      Plotly.newPlot(cell_for_plot, data, layout);
  }

    var active_pt = null;
    var all_pts = [];
    var ptR = 10.0;
    var debug_txt = "";

    function is_close(pt1, pt2) {
      return   (pt1[0] - pt2[0])*(pt1[0] - pt2[0])
            +  (pt1[1] - pt2[1])*(pt1[1] - pt2[1])
            <= ptR*ptR;
    }
  
    function circ(ctx, pos) {
      ctx.beginPath();
      if (pos[2] == 0) {
          ctx.fillStyle = 'rgba(0, 128, 0, 0.8)';
          ctx.arc(pos[0], pos[1], ptR, 0.0, 2.0 * Math.PI, 0);
          ctx.fill();
      } else {
          ctx.fillStyle = 'rgba(0, 208, 0, 1.0)'; //'rgba(0, 89, 208, 0.4)';
          ctx.fillRect(pos[0] - ptR, pos[1] - ptR, 2 * ptR, 2 * ptR);
          ctx.fillStyle = 'rgba(128, 0, 0, 1.0)';
          ctx.fillRect(pos[3] - ptR/2, pos[4] - ptR/2, ptR, ptR);
          ctx.fillRect(pos[5] - ptR/2, pos[6] - ptR/2, ptR, ptR);
          ctx.fillStyle = 'rgba(128, 0, 0, 0.1)';
          var rx = Math.sqrt((pos[4] - pos[1])*(pos[4] - pos[1]) + (pos[3] - pos[0])*(pos[3] - pos[0]));
          var ry = Math.sqrt((pos[6] - pos[1])*(pos[6] - pos[1]) + (pos[5] - pos[0])*(pos[5] - pos[0]));
          ctx.ellipse(pos[0], pos[1], rx, ry, Math.atan2(pos[4] - pos[1], pos[3] - pos[0]), 0, Math.PI * 2.0);
          ctx.fill();
      }
    }
    
    
    function draw() {
      var canvas = document.getElementById('paint');
      var plt_div = document.getElementById('for_right_plot');
      var el = plt_div.getElementsByClassName('plot')[0];
      var rect = el.getBoundingClientRect();
      canvas.style.cssText = "position:absolute; top:" + parseInt(rect.top)
                      + "px; left: " + parseInt(rect.left)
                      + "px; width:" + parseInt(rect.width)
                      + "px; height:" + parseInt(rect.height)
                      + "px; z-index:1000;";

      if (canvas.getContext) {
        var ctx = canvas.getContext('2d');
        
        ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
        ctx.strokeStyle = 'black';
        ctx.lineWidth = 4.0;
        ctx.strokeRect(0, 0, canvas.width, canvas.height); // field
        
        ctx.font = '20px serif';
        ctx.fillStyle = 'black';
        ctx.fillText(debug_txt, 200, 200);
        
        if (active_pt) {
          circ(ctx, active_pt);
        }
        all_pts.forEach(function (item, index) {
          circ(ctx, item);

        });
        //circ(ctx, all_pts[0]);


      }
    }
    
    var timer

    function mdown_handle(evt) {
      evt.stopPropagation();
      x = evt.offsetX;
      y = evt.offsetY;
      idx = -1;
      var item = all_pts[0];
      var index = 0;
      //all_pts.forEach(function (item, index) {
        if (is_close([x,y], [item[0], item[1]])) {
          idx = index;
        if (item[2] == 1)
            item[7] = 0;
        }
        if (item[2] == 1 && is_close([x,y], [item[3], item[4]])) {
          item[7] = 1;
          idx = index;
        }
        if (item[2] == 1 && is_close([x,y], [item[5], item[6]])) {
          item[7] = 2;
          idx = index;
        }
      //});
      if (idx <= -1)
          return;

    active_pt = all_pts[idx];
    all_pts.splice(idx, 1);
    draw();
    }
    
    function mmove_handle(evt) {
      evt.stopPropagation();
      x = evt.offsetX;
      y = evt.offsetY;
      if (active_pt) {
        if (active_pt[2] == 0) {
          active_pt[0] = x;
          active_pt[1] = y;
        }
        if (active_pt[2] == 1) {
          if (active_pt[7] == 0) {
              active_pt[3] += x - active_pt[0];
              active_pt[4] += y - active_pt[1];
              active_pt[5] += x - active_pt[0];
              active_pt[6] += y - active_pt[1];
              active_pt[0] = x;
            active_pt[1] = y;
            }
            if (active_pt[7] == 1) {
              active_pt[3] = x;
              active_pt[4] = y;
              var v_x = active_pt[3] - active_pt[0];
              var v_y = active_pt[4] - active_pt[1];
              var r = Math.sqrt(v_x*v_x + v_y*v_y);
              var u_x = active_pt[5] - active_pt[0];
              var u_y = active_pt[6] - active_pt[1];
              var R = Math.sqrt(u_x*u_x + u_y*u_y);
              active_pt[5] = active_pt[0] - v_y / r * R;
              active_pt[6] = active_pt[1] + v_x / r * R;
            }
            if (active_pt[7] == 2) {
              active_pt[5] = x;
              active_pt[6] = y;
              var v_x = active_pt[3] - active_pt[0];
              var v_y = active_pt[4] - active_pt[1];
              var r = Math.sqrt(v_x*v_x + v_y*v_y);
              var u_x = active_pt[5] - active_pt[0];
              var u_y = active_pt[6] - active_pt[1];
              var R = Math.sqrt(u_x*u_x + u_y*u_y);
              active_pt[3] = active_pt[0] + u_y / R * r;
              active_pt[4] = active_pt[1] - u_x / R * r;

            }         
        }
        //recalc_sse();
      }
      draw();
      plot_potentials();
      //update_3d();
    }
    
    function mup_handle(evt) {
      evt.stopPropagation();
      x = evt.offsetX;
      y = evt.offsetY;
      if (x >= 600) {
        active_pt = null;
      }
      if (active_pt) {
        //all_pts.push(active_pt);
        all_pts = [active_pt];
        active_pt = null;
        //recalc_sse();
        //regenerate();
      }
      draw();
      plot_potentials();
      //loss_function_3d();
    }

  </script>
</head>



<div id="for_right_plot">
</div>

<canvas id="paint" width=600 height=600
  onmousedown="mdown_handle(event)"
  onmousemove="mmove_handle(event)"
  onmouseup="mup_handle(event)">
</canvas>


<script>
function init()
{
  // setup point params
  active_pt = null;
  var x = 300;
  var y = 300;
  all_pts = [[x, y, 0]];
  plot_potentials();
  // setup canvas
  var plt_div = document.getElementById('for_right_plot');
  var el = plt_div.getElementsByClassName('plot')[0];
  var rect = el.getBoundingClientRect();
  var canvas = document.getElementById("paint");
  canvas.style.cssText = "position:absolute; top:" + parseInt(rect.top)
                      + "px; left: " + parseInt(rect.left)
                      + "px; width:" + parseInt(rect.width)
                      + "px; height:" + parseInt(rect.height)
                      + "px; z-index:1000;";
  canvas.width = rect.width;
  canvas.height = rect.height;
  var ctx = canvas.getContext('2d');
  ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
  draw();
}
init();
</script>

</html>
'''

import IPython
display(IPython.display.HTML(main_str))

#Exploring Newton's method

Newton's method is well-known method for minimization problems (many modifications are known).
Try the following demo with the simplest implementation of the Newton's method and gain some intuition on the following
* Does it always find minimum or maximum? Or both?
* How does it behave at the inflection points?
* How many iterations cdoes it need if the function is quadratic?
* Which problems of the simplest gradient and modified gradient from the previous examples does it fix?

In [37]:
#@title #Exploring Newton's method


from IPython.display import display
import ipywidgets as widgets


class Memory:
  def __init__(self):
    self.x_ini = 0.0
    self.y_ini = 0.0
    self.iters = 2

current_memory = Memory()

button = widgets.Button(description="Recalculate")
iters = widgets.IntSlider(min=2, max=50, value=current_memory.iters)

fnc = widgets.Text(value='sin(x) + 2 * sin(y)',
                   #placeholder='function',
                   description='Function:',
                   disabled=False)

def set_iter(val):
  current_memory.iters = val.new
iters.observe(set_iter, names='value')

display(widgets.HBox([button, fnc, iters]))

def on_button_clicked(b):
  function = fnc.value
  from IPython.display import clear_output
  clear_output()
  display(widgets.HBox([button, fnc, iters]))
  print('Doing Science...')

  x_ini = current_memory.x_ini
  y_ini = current_memory.y_ini
  max_iter = current_memory.iters


  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  import numpy as np
  from scipy.optimize import minimize

  from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
  transformations = (standard_transformations + (implicit_multiplication_application,))
  f = parse_expr(function, transformations=transformations)

  from sympy import diff
  from sympy import lambdify
  #g_x = lambda x,y: diff(f, 'x').evalf(subs={'x':x, 'y':y})
  #g_y = lambda x,y: diff(f, 'y').evalf(subs={'x':x, 'y':y})
  #jacobian = lambda x: np.array([g_x(x[0], x[1]), g_y(x[0], x[1])], dtype=float)
  g_x = lambdify(['x', 'y'], diff(f, 'x'), "numpy")
  g_y = lambdify(['x', 'y'], diff(f, 'y'), "numpy")
  jacobian = lambda x,y: np.array([g_x(x, y), g_y(x, y)], dtype=float)

  #g_xx = lambda x,y: diff(diff(f, 'x'), 'x').evalf(subs={'x':x, 'y':y})
  #g_xy = lambda x,y: diff(diff(f, 'x'), 'y').evalf(subs={'x':x, 'y':y})
  #g_yy = lambda x,y: diff(diff(f, 'y'), 'y').evalf(subs={'x':x, 'y':y})
  #H = lambda x,y: [[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]]
  #hessian = lambda x: np.array(H(x[0], x[1]), dtype=float)
  g_xx = lambdify(['x', 'y'], diff(diff(f, 'x'), 'x'), "numpy")
  g_xy = lambdify(['x', 'y'], diff(diff(f, 'x'), 'y'), "numpy")
  g_yy = lambdify(['x', 'y'], diff(diff(f, 'y'), 'y'), "numpy")
  hessian = lambda x,y: [[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]]

  #func_to_minimize = lambda x: float(f.evalf(subs={'x':x[0], 'y':x[1]}))
  func_to_minimize = lambdify(['x', 'y'], f, "numpy")

  history = [np.array([x_ini, y_ini])]
  crashed = False

  for i in range(max_iter):
    h = jacobian(history[-1][0], history[-1][1])
    H_ =  hessian(history[-1][0], history[-1][1])
    try:
      H_inv = np.linalg.inv(H_)
    except:
      crashed = True
      break
    theta = H_inv.dot(h)

    x_new = history[-1][0] - theta[0]
    y_new = history[-1][1] - theta[1]
    history.append(np.array([x_new, y_new], dtype='float'))

  _h = np.array(history)
  _x = _h.T[0]
  _y = _h.T[1]
  _z = np.array([func_to_minimize(x[0], x[1]) for x in history], dtype=float)

  x_min = min(-10.0, np.min(_x))
  x_max = max(10.0, np.max(_x))
  y_min = min(-10.0, np.min(_y))
  y_max = max(10.0, np.max(_y))

  x_ = np.linspace(x_min, x_max, num=50)
  y_ = np.linspace(y_min, y_max, num=50)
  z_ = np.array([[func_to_minimize(x,y) for x in x_] for y in y_], dtype=float)


  import matplotlib.pyplot as plt
  import base64
  import io
  fig = plt.figure(figsize=(20,20))
  ax = fig.gca()
  ax.axis('off')
  if np.max(z_) - np.min(z_) > 1000:
    ax.contourf(x_, y_, np.sign(z_)*np.log(np.abs(z_) + 1.0), alpha=0.3)  
  else:
    ax.contourf(x_, y_, z_, alpha=0.3)
  plt.close(fig)
  buf = io.BytesIO()
  fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0);
  image_base64 = u'data:image/  png;base64,' + base64.b64encode(buf.getvalue()).decode('utf-8').replace('\n', '')
  buf.close()


  fig = dict(
      layout = dict(
          width=1200, height=600, autosize=False,
          showlegend = False,
          scene = {'domain': { 'x': [0.0, 0.44], 'y': [0, 1] } },
          xaxis1 = {'domain': [0.55, 1], 'range': [x_min, x_max], 'fixedrange': True},
          yaxis1 = {'domain': [0.0, 1.0], 'range': [y_min, y_max], 'fixedrange': True},
          title  = 'Minimization',
          margin = {'t': 50, 'b': 50, 'l': 50, 'r': 50},
          updatemenus = [{'buttons': [{'args': [[k for k in range(len(_h)-1)],
                                                {'frame': {'duration': 500.0, 'redraw': True},
                                                'fromcurrent': False, 'transition': {'duration': 0, 'easing': 'linear'}}],
                                      'label': 'Play',
                                      'method': 'animate'},
                                      {'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                                        'mode': 'immediate',
                                                        'transition': {'duration': 0}}
                                                ],
                                      'label': 'Pause',
                                      'method': 'animate'
                                      }
                                      ],
                          'direction': 'left',
                          'pad': {'r': 10, 't': 85},
                          'showactive': True,
                          'type': 'buttons', 'x': 0.1, 'y': 0, 'xanchor': 'right', 'yanchor': 'top'}],
          sliders = [{'yanchor': 'top',
                      'xanchor': 'left',
                      'currentvalue': {'font': {'size': 16}, 'prefix': 'Step: ', 'visible': True, 'xanchor': 'right'},
                      'transition': {'duration': 0.0},
                      'pad': {'b': 10, 't': 50},
                      'len': 0.9,
                      'x': 0.1,
                      'y': 0,
                      'steps': [{'args': [[k], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': True},
                                                'transition': {'duration': 0, 'easing': 'linear'}}
                                          ],
                                'label': k,
                                'method': 'animate'} for k in range(len(_h)-1)
                      ]}],
          images = [{'source' : image_base64,
                    'xref': 'x', 'yref': 'y',
                    'sizing': 'stretch',
                    'sizex': x_max - x_min, 'sizey': y_max - y_min,
                    'layer': 'below', 'opacity':1.0,
                    'x': x_min, 'y': y_max}]
      ),
      data = [
          {'type': 'scatter3d', 'name': 's3', 'x': _x, 'y': _y, 'z': _z, 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
          {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
          #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
          {'type': 'scatter', 'name': 's2', 'x': _x, 'y': _y, 
          'line': {'color': 'red', 'width': 2}
          },
      ],
      frames=[
          {'name': str(k),
          'data': [
            {'type': 'scatter3d', 'name': 's3', 'x': _x[:k], 'y': _y[:k], 'z': _z[:k], 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
            {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
            #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
            {'type': 'scatter', 'name': 's2', 'x': _x[:k], 'y': _y[:k], 
            'line': {'color': 'red', 'width': 2}
            },
          ]} for k in range(len(_h)-1) ]
  )
  #plot(fig, auto_open=False)
  clear_output()
  if (crashed):
      print("Cannot find inverse matrix, Method compromised")
      print("Current Hessian value: ")
      print(H_)
      print("Current point: ", history[-1])

  display(widgets.HBox([button, fnc, iters]))
  f = go.Figure(fig)
  f.show()

  def save_pos(pos):
    global current_memory
    current_memory.x_ini = (x_max - x_min) * pos[0] + x_min
    current_memory.y_ini = (y_max - y_min) * (1.0 - pos[1]) + y_min

  main_str = '''
  <canvas id="paint_here"
          onmousedown="mdown_handle(event)"
          onmousemove="mmove_handle(event)"
          onmouseup="mup_handle(event)"></canvas>
  <script>

  var el = document.getElementsByClassName("layer-subplot")[0];
  var rect = el.getBoundingClientRect();

  var canvas = document.getElementById("paint_here");
  canvas.style.cssText = "position:absolute; top:" + rect.top
                      + "px; left: " + rect.left
                      + "px; width:" + rect.width
                      + "px; height:" + rect.height
                      + "px; z-index:1000;";
  canvas.width = rect.width;
  canvas.height = rect.height;
  var ctx = canvas.getContext('2d');
  ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
  //ctx.fillStyle="#00FF00";
  //ctx.fillRect(0, 0, canvas.width, canvas.height); // field
  ''' + 'var x_ini = ' + str((current_memory.x_ini - x_min)/(x_max - x_min)) + ';' + 'var y_ini = ' + str(1.0 - (current_memory.y_ini - y_min)/(y_max - y_min)) + ';' + '''
  var active_pt = [canvas.width * x_ini, canvas.height * y_ini];

  function draw() {
      ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
      //ctx.fillText("drawing", 20, 20);

      ctx.beginPath();
      ctx.arc(active_pt[0], active_pt[1], 10, 0.0, 2.0 * Math.PI, 0);
      ctx.fillStyle = "rgba(210, 0, 0, 0.75)";
      ctx.fill();
  }

  var do_move = false;

  function is_close(pt1, pt2) {
    return   (pt1[0] - pt2[0])*(pt1[0] - pt2[0])
          +  (pt1[1] - pt2[1])*(pt1[1] - pt2[1])
          <= 10*10;
  }

  function mdown_handle(evt) {
    x = evt.offsetX;
    y = evt.offsetY;
    do_move = is_close(active_pt, [x, y]);
  }
      
  function mmove_handle(evt) {
    if (!do_move)
        return;
    active_pt[0] = evt.offsetX;
    active_pt[1] = evt.offsetY;
  }
      
  function mup_handle(evt) {
    do_move = false;
    remember();
  }

  var w = canvas.width;
  var h = canvas.height;

  async function remember() {
    var x = active_pt[0] / w;
    var y = active_pt[1] / h;
    const result = await google.colab.kernel.invokeFunction('notebook.rememberPos', [[x, y]], {});
  }

  var timer = setInterval(draw, 10);

  </script>
  '''

  import IPython
  from google.colab import output
  display(IPython.display.HTML(main_str))
  output.register_callback('notebook.rememberPos', save_pos)


button.on_click(on_button_clicked)

#Exploring minimization methods from scikit-learn

Explore different minimization methods available through the function `minimize`.
Try to find out their strengths and weaknesses.
Try different functions and initial conditions.
If the method needs you to calculate gradient (jacobian) and Hessian matrix -- set corresponding values to 'custom' (in contrary to other options this is NOT an option of scikit-learn -- I implemented that for you using sympy).
Consult [scipy.optimize.minimize help](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) to undestand parameters better.

In [59]:
#@title #Exploring minimization methods


from IPython.display import display
import ipywidgets as widgets

possible_methods = ['Nelder-Mead','Powell','CG','BFGS','Newton-CG','L-BFGS-B','TNC','COBYLA','SLSQP','trust-constr','dogleg','trust-ncg','trust-exact','trust-krylov']
possible_jacs = ['none', '2-point', '3-point', 'cs', 'custom']
possible_hess = ['none', '2-point', '3-point', 'cs', 'custom']

class Memory:
  def __init__(self):
    self.x_ini = 0.0
    self.y_ini = 0.0
    self.method = 'Nelder-Mead'
    self.jac = 'none'
    self.hess = 'none'
    self.iters = 2
    self.function = 'x**2+sin(y)*20'

current_memory = Memory()

#if not 'current_memory' in globals():
#  global current_memory
#  current_memory = Memory()
#
#for attr in ['iters', 'jac', 'hess', 'method', 'x_ini', 'y_ini']:
#  if not hasattr(current_memory, attr):
#    current_memory = Memory()

button = widgets.Button(description="Recalculate")
iters = widgets.IntSlider(min=2, max=50, value=current_memory.iters)
method_widg = widgets.Dropdown(options=possible_methods,
                          value=current_memory.method,
                          description='Method:',
                           disabled=False)
jac = widgets.Dropdown(options=possible_jacs,
                       value=current_memory.jac,
                       description='Jacobian:',
                       disabled=False)
hess = widgets.Dropdown(options=possible_hess,
                        value=current_memory.hess,
                        description='Hessian:',
                        disabled=False)
fnc = widgets.Text(value=current_memory.function,
                   #placeholder='function',
                   description='Function:',
                   disabled=False)

def set_method(val):
  current_memory.method = val.new
method_widg.observe(set_method, names='value')
def set_jac(val):
  current_memory.jac = val.new
jac.observe(set_jac, names='value')
def set_hess(val):
  current_memory.hess = val.new
hess.observe(set_hess, names='value')
def set_iter(val):
  current_memory.iters = val.new
iters.observe(set_iter, names='value')

display(widgets.HBox([button, fnc, iters, method_widg, jac, hess]))

def on_button_clicked(b):
  function = fnc.value
  from IPython.display import clear_output
  clear_output()
  display(widgets.HBox([button, fnc, iters, method_widg, jac, hess]))
  print('Doing Science...')

  x_ini = current_memory.x_ini
  y_ini = current_memory.y_ini
  method = current_memory.method
  max_iter = current_memory.iters
  jacobian = current_memory.jac
  hessian = current_memory.hess


  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  import numpy as np
  from scipy.optimize import minimize

  from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
  transformations = (standard_transformations + (implicit_multiplication_application,))
  f = parse_expr(function, transformations=transformations)

  from sympy import diff
  from sympy import lambdify
  #g_x = lambda x,y: diff(f, 'x').evalf(subs={'x':x, 'y':y})
  #g_y = lambda x,y: diff(f, 'y').evalf(subs={'x':x, 'y':y})
  g_x = lambdify(['x', 'y'], diff(f, 'x'), "numpy")
  g_y = lambdify(['x', 'y'], diff(f, 'y'), "numpy")

  if jacobian == 'custom':
    jacobian = lambda x: np.array([g_x(x[0], x[1]), g_y(x[0], x[1])], dtype=float)
  if jacobian == 'none':
    jacobian = None

  #g_xx = lambda x,y: diff(diff(f, 'x'), 'x').evalf(subs={'x':x, 'y':y})
  #g_xy = lambda x,y: diff(diff(f, 'x'), 'y').evalf(subs={'x':x, 'y':y})
  #g_yy = lambda x,y: diff(diff(f, 'y'), 'y').evalf(subs={'x':x, 'y':y})
  g_xx = lambdify(['x', 'y'], diff(diff(f, 'x'), 'x'), "numpy")
  g_xy = lambdify(['x', 'y'], diff(diff(f, 'x'), 'y'), "numpy")
  g_yy = lambdify(['x', 'y'], diff(diff(f, 'y'), 'y'), "numpy")
  H = lambda x,y: [[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]]

  if hessian == 'custom':
    hessian = lambda x: np.array(H(x[0], x[1]), dtype=float)
  if hessian == 'none':
    hessian = None

  ftm = lambdify(['x', 'y'], f, "numpy")
  func_to_minimize = lambda x: ftm(x[0], x[1]) #float(f.evalf(subs={'x':x[0], 'y':x[1]}))

  history = [np.array([x_ini, y_ini])]
  trust_radii = []
  def store_data(xk, *args): 
    if len(args) > 0 and 'tr_radius' in args[0]:
      pass
      #print(args[0]['tr_radius'])
      trust_radii.append(args[0]['tr_radius'])
    history.append(xk)

  minimize(func_to_minimize, [x_ini, y_ini], 
          method=method, jac=jacobian, hess=hessian,
          options={'maxiter':max_iter}, callback=store_data)

  _h = np.array(history)
  _x = _h.T[0]
  _y = _h.T[1]
  _z = np.array([func_to_minimize(x) for x in history], dtype=float)

  x_min = min(-10.0, np.min(_x))
  x_max = max(10.0, np.max(_x))
  y_min = min(-10.0, np.min(_y))
  y_max = max(10.0, np.max(_y))

  x_ = np.linspace(x_min, x_max, num=50)
  y_ = np.linspace(y_min, y_max, num=50)
  z_ = np.array([[func_to_minimize([x,y]) for x in x_] for y in y_], dtype=float)

  dz = np.max(z_) - np.min(z_)

  import matplotlib.pyplot as plt
  import base64
  import io
  fig = plt.figure(figsize=(20,20))
  ax = fig.gca()
  ax.axis('off')
  if dz > 1000:
    ax.contourf(x_, y_, np.sign(z_)*np.log(np.abs(z_) + 1.0), alpha=0.3)  
  else:
    ax.contourf(x_, y_, z_, alpha=0.3)
  plt.close(fig)
  buf = io.BytesIO()
  fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0);
  image_base64 = u'data:image/  png;base64,' + base64.b64encode(buf.getvalue()).decode('utf-8').replace('\n', '')
  buf.close()


  fig = dict(
      layout = dict(
          width=1200, height=600, autosize=False,
          showlegend = False,
          scene = {'domain': { 'x': [0.0, 0.44], 'y': [0, 1] } },
          xaxis1 = {'domain': [0.55, 1], 'range': [x_min, x_max], 'fixedrange': True},
          yaxis1 = {'domain': [0.0, 1.0], 'range': [y_min, y_max], 'fixedrange': True},
          title  = 'Minimization',
          margin = {'t': 50, 'b': 50, 'l': 50, 'r': 50},
          updatemenus = [{'buttons': [{'args': [[k for k in range(len(_h)-1)],
                                                {'frame': {'duration': 500.0, 'redraw': True},
                                                'fromcurrent': False, 'transition': {'duration': 0, 'easing': 'linear'}}],
                                      'label': 'Play',
                                      'method': 'animate'},
                                      {'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                                        'mode': 'immediate',
                                                        'transition': {'duration': 0}}
                                                ],
                                      'label': 'Pause',
                                      'method': 'animate'
                                      }
                                      ],
                          'direction': 'left',
                          'pad': {'r': 10, 't': 85},
                          'showactive': True,
                          'type': 'buttons', 'x': 0.1, 'y': 0, 'xanchor': 'right', 'yanchor': 'top'}],
          sliders = [{'yanchor': 'top',
                      'xanchor': 'left',
                      'currentvalue': {'font': {'size': 16}, 'prefix': 'Step: ', 'visible': True, 'xanchor': 'right'},
                      'transition': {'duration': 0.0},
                      'pad': {'b': 10, 't': 50},
                      'len': 0.9,
                      'x': 0.1,
                      'y': 0,
                      'steps': [{'args': [[k], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': True},
                                                'transition': {'duration': 0, 'easing': 'linear'}}
                                          ],
                                'label': k,
                                'method': 'animate'} for k in range(len(_h)-1)
                      ]}],
          images = [{'source' : image_base64,
                    'xref': 'x', 'yref': 'y',
                    'sizing': 'stretch',
                    'sizex': x_max - x_min, 'sizey': y_max - y_min,
                    'layer': 'below', 'opacity':1.0,
                    'x': x_min, 'y': y_max}]
      ),
      data = [
          {'type': 'scatter3d', 'name': 's3', 'x': _x, 'y': _y, 'z': _z, 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
          {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
          #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
          {'type': 'scatter', 'name': 's2', 'x': _x, 'y': _y, 
          'line': {'color': 'red', 'width': 2}
          },
          {
              'type': 'scatter', 'name': 'trust radii', 
              'x': _x, 'y': _y, 'mode': 'markers',
              'marker': {'size': trust_radii, 
                         'sizeref': 0.05,
                         'color': 'rgba(0, 0, 0, 0)',
                         'line': {'color': 'red', 'width':2}},
          } if len(trust_radii) > 0 else {}
      ],
      frames=[
          {'name': str(k),
          'data': [
            {'type': 'scatter3d', 'name': 's3', 'x': _x[:k], 'y': _y[:k], 'z': _z[:k], 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
            {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
            #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
            {'type': 'scatter', 'name': 's2', 'x': _x[:k], 'y': _y[:k], 
            'line': {'color': 'red', 'width': 2}
            },
            {
              'type': 'scatter', 'name': 'trust radii', 
              'x': _x[:k], 'y': _y[:k], 'mode': 'markers',
              'marker': {'size': trust_radii[:k], 
                         'sizeref': 0.05,
                         'color': 'rgba(0, 0, 0, 0)',
                         'line': {'color': 'red', 'width':2}}
            } if len(trust_radii) > 0 else {}
          ]} for k in range(len(_h)-1) ]
  )
  #plot(fig, auto_open=False)

  clear_output()
  display(widgets.HBox([button, fnc, iters, method_widg, jac, hess]))
  f = go.Figure(fig)
  f.show()

  def save_pos(pos):
    global current_memory
    current_memory.x_ini = (x_max - x_min) * pos[0] + x_min
    current_memory.y_ini = (y_max - y_min) * (1.0 - pos[1]) + y_min

  main_str = '''
  <canvas id="paint_here"
          onmousedown="mdown_handle(event)"
          onmousemove="mmove_handle(event)"
          onmouseup="mup_handle(event)"></canvas>
  <script>

  var el = document.getElementsByClassName("layer-subplot")[0];
  var rect = el.getBoundingClientRect();

  var canvas = document.getElementById("paint_here");
  canvas.style.cssText = "position:absolute; top:" + rect.top
                      + "px; left: " + rect.left
                      + "px; width:" + rect.width
                      + "px; height:" + rect.height
                      + "px; z-index:1000;";
  canvas.width = rect.width;
  canvas.height = rect.height;
  var ctx = canvas.getContext('2d');
  ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
  //ctx.fillStyle="#00FF00";
  //ctx.fillRect(0, 0, canvas.width, canvas.height); // field
  ''' + 'var x_ini = ' + str((current_memory.x_ini - x_min)/(x_max - x_min)) + ';' + 'var y_ini = ' + str(1.0 - (current_memory.y_ini - y_min)/(y_max - y_min)) + ';' + '''
  var active_pt = [canvas.width * x_ini, canvas.height * y_ini];

  function draw() {
      ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
      //ctx.fillText("drawing", 20, 20);

      ctx.beginPath();
      ctx.arc(active_pt[0], active_pt[1], 10, 0.0, 2.0 * Math.PI, 0);
      ctx.fillStyle = "rgba(210, 0, 0, 0.75)";
      ctx.fill();
  }

  var do_move = false;

  function is_close(pt1, pt2) {
    return   (pt1[0] - pt2[0])*(pt1[0] - pt2[0])
          +  (pt1[1] - pt2[1])*(pt1[1] - pt2[1])
          <= 10*10;
  }

  function mdown_handle(evt) {
    x = evt.offsetX;
    y = evt.offsetY;
    do_move = is_close(active_pt, [x, y]);
  }
      
  function mmove_handle(evt) {
    if (!do_move)
        return;
    active_pt[0] = evt.offsetX;
    active_pt[1] = evt.offsetY;
  }
      
  function mup_handle(evt) {
    do_move = false;
    remember();
  }

  var w = canvas.width;
  var h = canvas.height;

  async function remember() {
    var x = active_pt[0] / w;
    var y = active_pt[1] / h;
    const result = await google.colab.kernel.invokeFunction('notebook.rememberPos', [[x, y]], {});
  }

  var timer = setInterval(draw, 10);

  </script>
  '''

  import IPython
  from google.colab import output
  display(IPython.display.HTML(main_str))
  output.register_callback('notebook.rememberPos', save_pos)


button.on_click(on_button_clicked)

#Implementing your own minimization method

Implement your own minimization method.
Visualization and iteration are already implemented, only the core function `next_step` is needed.
Modify it to get different minimization methods, use lecture notes if you need formulas.
Point $\vec{x}$, function $f(\vec{x})$, its gradient and Hessian matrix are already passed to `next_step` as parameters.
If your method requires additional parameters -- modify `initial_additional_args` to store them.

In [52]:
import numpy as np

# put here all additional arguments you need
# initialize with values you need at the first iteration
# update them on function call as needed
# and return them as function finishes
initial_additional_args = {'step': 0}

# the following function takes as input
# x -- numpy array [x,y] that is current point vector
# f -- value of the function at x
# g -- gradient vector (numpy array) [g_x, g_y]
# h -- hessian matrix (numpy array) [[g_xx, g_xy], 
#                                    [g_xy, g_yy]]
def next_step(x, f, g, h, additional_args):
  theta = 0.1
  additional_args['step'] += 1
  if additional_args['step'] == 20: # long jump on 20-th step
    x += np.array([5,5])
  return x - theta * g, additional_args

In [56]:
#@title #Implementing your own minimization method


from IPython.display import display
import ipywidgets as widgets


class Memory:
  def __init__(self):
    self.x_ini = 0.0
    self.y_ini = 0.0
    self.iters = 2

current_memory = Memory()

button = widgets.Button(description="Recalculate")
iters = widgets.IntSlider(min=2, max=50, value=current_memory.iters)

fnc = widgets.Text(value='x**2+sin(y)*20',
                   #placeholder='function',
                   description='Function:',
                   disabled=False)

def set_iter(val):
  current_memory.iters = val.new
iters.observe(set_iter, names='value')

display(widgets.HBox([button, fnc, iters]))

def on_button_clicked(b):
  function = fnc.value
  from IPython.display import clear_output
  clear_output()
  display(widgets.HBox([button, fnc, iters]))
  print('Doing Science...')

  x_ini = current_memory.x_ini
  y_ini = current_memory.y_ini
  max_iter = current_memory.iters


  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  import numpy as np
  from scipy.optimize import minimize

  from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
  transformations = (standard_transformations + (implicit_multiplication_application,))
  f = parse_expr(function, transformations=transformations)

  from sympy import diff
  from sympy import lambdify
  #g_x = lambda x,y: diff(f, 'x').evalf(subs={'x':x, 'y':y})
  #g_y = lambda x,y: diff(f, 'y').evalf(subs={'x':x, 'y':y})
  g_x = lambdify(['x', 'y'], diff(f, 'x'), "numpy")
  g_y = lambdify(['x', 'y'], diff(f, 'y'), "numpy")
  jacobian = lambda x: np.array([g_x(x[0], x[1]), g_y(x[0], x[1])], dtype=float)


  #g_xx = lambda x,y: diff(diff(f, 'x'), 'x').evalf(subs={'x':x, 'y':y})
  #g_xy = lambda x,y: diff(diff(f, 'x'), 'y').evalf(subs={'x':x, 'y':y})
  #g_yy = lambda x,y: diff(diff(f, 'y'), 'y').evalf(subs={'x':x, 'y':y})
  g_xx = lambdify(['x', 'y'], diff(diff(f, 'x'), 'x'), "numpy")
  g_xy = lambdify(['x', 'y'], diff(diff(f, 'x'), 'y'), "numpy")
  g_yy = lambdify(['x', 'y'], diff(diff(f, 'y'), 'y'), "numpy")
  H = lambda x,y: [[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]]
  hessian = lambda x: np.array(H(x[0], x[1]), dtype=float)

  ftm = lambdify(['x', 'y'], f, "numpy")
  func_to_minimize = lambda x: ftm(x[0], x[1]) #float(f.evalf(subs={'x':x[0], 'y':x[1]}))
  #func_to_minimize = lambda x: float(f.evalf(subs={'x':x[0], 'y':x[1]}))
  history = [np.array([x_ini, y_ini])]
  def store_data(xk, *args): 
    history.append(xk) 


  a = initial_additional_args.copy()
  for step in range(max_iter):
    x_next, a = next_step(history[-1], func_to_minimize(history[-1]),
                          jacobian(history[-1]), hessian(history[-1]), a)
    store_data(x_next)


  _h = np.array(history)
  _x = _h.T[0]
  _y = _h.T[1]
  _z = np.array([func_to_minimize(x) for x in history], dtype=float)

  x_min = min(-10.0, np.min(_x))
  x_max = max(10.0, np.max(_x))
  y_min = min(-10.0, np.min(_y))
  y_max = max(10.0, np.max(_y))

  x_ = np.linspace(x_min, x_max, num=50)
  y_ = np.linspace(y_min, y_max, num=50)
  z_ = np.array([[func_to_minimize([x,y]) for x in x_] for y in y_], dtype=float)


  import matplotlib.pyplot as plt
  import base64
  import io
  fig = plt.figure(figsize=(20,20))
  ax = fig.gca()
  ax.axis('off')
  if np.max(z_) - np.min(z_) > 1000:
    ax.contourf(x_, y_, np.sign(z_)*np.log(np.abs(z_) + 1.0), alpha=0.3)  
  else:
    ax.contourf(x_, y_, z_, alpha=0.3)
  plt.close(fig)
  buf = io.BytesIO()
  fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0);
  image_base64 = u'data:image/  png;base64,' + base64.b64encode(buf.getvalue()).decode('utf-8').replace('\n', '')
  buf.close()


  fig = dict(
      layout = dict(
          width=1200, height=600, autosize=False,
          showlegend = False,
          scene = {'domain': { 'x': [0.0, 0.44], 'y': [0, 1] } },
          xaxis1 = {'domain': [0.55, 1], 'range': [x_min, x_max], 'fixedrange': True},
          yaxis1 = {'domain': [0.0, 1.0], 'range': [y_min, y_max], 'fixedrange': True},
          title  = 'Minimization',
          margin = {'t': 50, 'b': 50, 'l': 50, 'r': 50},
          updatemenus = [{'buttons': [{'args': [[k for k in range(len(_h)-1)],
                                                {'frame': {'duration': 500.0, 'redraw': True},
                                                'fromcurrent': False, 'transition': {'duration': 0, 'easing': 'linear'}}],
                                      'label': 'Play',
                                      'method': 'animate'},
                                      {'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                                        'mode': 'immediate',
                                                        'transition': {'duration': 0}}
                                                ],
                                      'label': 'Pause',
                                      'method': 'animate'
                                      }
                                      ],
                          'direction': 'left',
                          'pad': {'r': 10, 't': 85},
                          'showactive': True,
                          'type': 'buttons', 'x': 0.1, 'y': 0, 'xanchor': 'right', 'yanchor': 'top'}],
          sliders = [{'yanchor': 'top',
                      'xanchor': 'left',
                      'currentvalue': {'font': {'size': 16}, 'prefix': 'Step: ', 'visible': True, 'xanchor': 'right'},
                      'transition': {'duration': 0.0},
                      'pad': {'b': 10, 't': 50},
                      'len': 0.9,
                      'x': 0.1,
                      'y': 0,
                      'steps': [{'args': [[k], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': True},
                                                'transition': {'duration': 0, 'easing': 'linear'}}
                                          ],
                                'label': k,
                                'method': 'animate'} for k in range(len(_h)-1)
                      ]}],
          images = [{'source' : image_base64,
                    'xref': 'x', 'yref': 'y',
                    'sizing': 'stretch',
                    'sizex': x_max - x_min, 'sizey': y_max - y_min,
                    'layer': 'below', 'opacity':1.0,
                    'x': x_min, 'y': y_max}]
      ),
      data = [
          {'type': 'scatter3d', 'name': 's3', 'x': _x, 'y': _y, 'z': _z, 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
          {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
          #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
          {'type': 'scatter', 'name': 's2', 'x': _x, 'y': _y, 
          'line': {'color': 'red', 'width': 2}
          },
          #{
          #    'type': 'scatter', 'name': 'trust radii', 
          #    'x': _x, 'y': _y, 'mode': 'markers',
          #    'marker': {'size': trust_radii, 'sizeref': 0.05}
          #} if len(trust_radii) > 0 else {}
      ],
      frames=[
          {'name': str(k),
          'data': [
            {'type': 'scatter3d', 'name': 's3', 'x': _x[:k], 'y': _y[:k], 'z': _z[:k], 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
            {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
            #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
            {'type': 'scatter', 'name': 's2', 'x': _x[:k], 'y': _y[:k], 
            'line': {'color': 'red', 'width': 2}
            },
          ]} for k in range(len(_h)-1) ]
  )
  #plot(fig, auto_open=False)
  clear_output()
  display(widgets.HBox([button, fnc, iters]))
  f = go.Figure(fig)
  f.show()

  def save_pos(pos):
    global current_memory
    current_memory.x_ini = (x_max - x_min) * pos[0] + x_min
    current_memory.y_ini = (y_max - y_min) * (1.0 - pos[1]) + y_min

  main_str = '''
  <canvas id="paint_here"
          onmousedown="mdown_handle(event)"
          onmousemove="mmove_handle(event)"
          onmouseup="mup_handle(event)"></canvas>
  <script>

  var el = document.getElementsByClassName("layer-subplot")[0];
  var rect = el.getBoundingClientRect();

  var canvas = document.getElementById("paint_here");
  canvas.style.cssText = "position:absolute; top:" + rect.top
                      + "px; left: " + rect.left
                      + "px; width:" + rect.width
                      + "px; height:" + rect.height
                      + "px; z-index:1000;";
  canvas.width = rect.width;
  canvas.height = rect.height;
  var ctx = canvas.getContext('2d');
  ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
  //ctx.fillStyle="#00FF00";
  //ctx.fillRect(0, 0, canvas.width, canvas.height); // field
  ''' + 'var x_ini = ' + str((current_memory.x_ini - x_min)/(x_max - x_min)) + ';' + 'var y_ini = ' + str(1.0 - (current_memory.y_ini - y_min)/(y_max - y_min)) + ';' + '''
  var active_pt = [canvas.width * x_ini, canvas.height * y_ini];

  function draw() {
      ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
      //ctx.fillText("drawing", 20, 20);

      ctx.beginPath();
      ctx.arc(active_pt[0], active_pt[1], 10, 0.0, 2.0 * Math.PI, 0);
      ctx.fillStyle = "rgba(210, 0, 0, 0.75)";
      ctx.fill();
  }

  var do_move = false;

  function is_close(pt1, pt2) {
    return   (pt1[0] - pt2[0])*(pt1[0] - pt2[0])
          +  (pt1[1] - pt2[1])*(pt1[1] - pt2[1])
          <= 10*10;
  }

  function mdown_handle(evt) {
    x = evt.offsetX;
    y = evt.offsetY;
    do_move = is_close(active_pt, [x, y]);
  }
      
  function mmove_handle(evt) {
    if (!do_move)
        return;
    active_pt[0] = evt.offsetX;
    active_pt[1] = evt.offsetY;
  }
      
  function mup_handle(evt) {
    do_move = false;
    remember();
  }

  var w = canvas.width;
  var h = canvas.height;

  async function remember() {
    var x = active_pt[0] / w;
    var y = active_pt[1] / h;
    const result = await google.colab.kernel.invokeFunction('notebook.rememberPos', [[x, y]], {});
  }

  var timer = setInterval(draw, 10);

  </script>
  '''

  import IPython
  from google.colab import output
  display(IPython.display.HTML(main_str))
  output.register_callback('notebook.rememberPos', save_pos)


button.on_click(on_button_clicked)

In [ ]:
initial_additional_args

{'step': 100}